In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as st

In [2]:
movie_budget = pd.read_csv('Files/tn.movie_budgets.csv.gz')
movie_budget
#import csv as a read me file for analysis

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [3]:
movie_budget.isnull().sum()
#checking to see if there are any null values in the data set for data cleaning

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [4]:
movie_budget['production_budget'] = movie_budget['production_budget'].str.replace('$','')

movie_budget['domestic_gross'] = movie_budget['domestic_gross'].str.replace('$','')

movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].str.replace('$','')
#getting rid of the dollar signs for numbers to turn them into floats to be able to compare numbers

In [5]:
movie_budget['production_budget'] = movie_budget['production_budget'].str.replace(',','')

movie_budget['domestic_gross'] = movie_budget['domestic_gross'].str.replace(',','')

movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].str.replace(',','')
#getting rid of the commas for numbers to turn them into floats to be able to compare numbers

In [6]:
movie_budget.worldwide_gross = movie_budget.worldwide_gross.astype(float)
movie_budget.domestic_gross = movie_budget.domestic_gross.astype(float)
movie_budget.production_budget = movie_budget.production_budget.astype(float)
#turning all of the number based columns into floats after taking out the commas and dollar signs

In [7]:
movie_budget.info()
#checking to make sure that the columns were converted from objects to integers for further analysis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


In [8]:
movie_budget.loc[movie_budget['worldwide_gross'] > movie_budget['production_budget'], 'Profitability'] = True
movie_budget.loc[movie_budget['worldwide_gross'] < movie_budget['production_budget'], 'Profitability'] = False
movie_budget.loc[movie_budget['worldwide_gross'] == movie_budget['production_budget'], 'Profitability'] = 'Broke Even'
#Creating a new column called profitability to see which movies made more than they spent by using inequality symbols
#Using true and false to easily see which movies were profitable

In [9]:
movie_budget['WW Profit ($)'] = movie_budget['worldwide_gross'] - movie_budget['production_budget']
#Creating a new column called WW Profit ($) by subtracting the production_budget away from the worldwide gross to find net
#this column factualizes the profitability column

In [10]:
movie_budget = movie_budget.sort_values(['WW Profit ($)'], ascending=False)
#sorting the WW profit column so that the most profitable overall are at the top to analyse if that effects our business
#reccomendations

In [11]:
movie_budget['WW Profit (millions)'] = '$' + (movie_budget['WW Profit ($)'].astype(float)/1000000).astype(str) + 'MM'
#creating a column that is the WW Profit ($) colum, but in millions with dollar signs so its more readable

In [12]:
movie_budget['Dom. Profit ($)'] = movie_budget['domestic_gross'] - movie_budget['production_budget']
##sorting the Domestic profit column so that the most profitable overall are at the top to analyse if that effects our business
#reccomendations

In [13]:
movie_budget['Dom. Profit (millions)'] = '$' + (movie_budget['Dom. Profit ($)'].astype(float)/1000000).astype(str) + 'MM'
#creating a column that is the Domestic Profit ($) colum, but in millions with dollar signs so its more readable

In [14]:
tmdb1 = pd.read_csv('Files/tmdb.movies.csv.gz')
#import csv as read me file 

In [15]:
tmdb1.rename({'title': 'movie'}, axis =1, inplace = True)
#renaming the title column in the tmdb1 file to movie so I can merge them

In [16]:
innerjoin = pd.merge(movie_budget,tmdb1, how='inner', on = 'movie')
#merging the two files on the column 'movie'

In [17]:
gen_title_pop = innerjoin[['genre_ids','movie', 'popularity','Dom. Profit (millions)', 'WW Profit (millions)']]
#titling a table with all the data I want it to show for analysis

In [18]:
gen_title_pop60 = gen_title_pop.loc[gen_title_pop['popularity']>=31.613].sort_values('popularity', ascending=False)
#looking at movies that only have a popularity over 31.613 (top 60)

In [19]:
gen_title_pop60['first_genre_id'] = [int(row.split(',')[0].lstrip('[').rstrip(']')) for row in gen_title_pop60['genre_ids']]
#spliting the genre column to give me the first genre listed so I can create a new genre column

In [20]:
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==12, "first_genre_id"] = 'Adventure'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==28, "first_genre_id"] = 'Action'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==878, "first_genre_id"] = 'Sci Fi'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==10751, "first_genre_id"] = 'Family'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==53, "first_genre_id"] = 'Thriller'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==18, "first_genre_id"] = 'Drama'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==14, "first_genre_id"] = 'Fantasy'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==80, "first_genre_id"] = 'Crime'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==36, "first_genre_id"] = 'History'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==35, "first_genre_id"] = 'Comedy'
gen_title_pop60.loc[gen_title_pop60["first_genre_id"]==16, "first_genre_id"] = 'Animation'
#creating new genre column to reflex what the first genre number is in words

In [21]:
num_gen = gen_title_pop60['first_genre_id'].value_counts()
num_gen
#counting how many of each there are in the top 60 most popular movies

Action       24
Adventure    10
Sci Fi        8
Drama         3
Family        2
Comedy        1
History       1
Crime         1
Thriller      1
Name: first_genre_id, dtype: int64

In [22]:
gen_title_pop60.sort_values('WW Profit (millions)', ascending = False).head(10)
#sorting the top 10 by worldwide profit to vizualize

,genre_ids,movie,popularity,Dom. Profit (millions),WW Profit (millions),first_genre_id
21,"[28, 12, 14]",Aquaman,38.102,$175.061807MM,$986.89464MM,Action
29,"[12, 28, 878]",Captain America: Civil War,39.137,$158.084349MM,$890.069413MM,Adventure
36,"[18, 10402]",Bohemian Rhapsody,37.197,$161.303339MM,$839.985342MM,Drama
49,"[28, 12, 35]",Deadpool,35.067,$305.070709MM,$743.025593MM,Action
636,"[878, 28, 53]",Blade Runner 2049,48.571,$-92.945841MM,$74.357408MM,Sci Fi
635,"[878, 28, 53]",Blade Runner 2049,48.571,$-92.945841MM,$74.357408MM,Sci Fi
51,"[878, 28]",Venom,44.007,$97.511408MM,$737.628605MM,Sci Fi
56,"[28, 12, 878, 18]",Spider-Man: Homecoming,46.775,$159.20114MM,$705.16635MM,Action
57,"[28, 12, 14]",The Hobbit: The Battle of the Five Armies,53.783,$5.119788MM,$695.577621MM,Action
60,"[28, 35, 12]",Deadpool 2,38.894,$214.591735MM,$676.680557MM,Action


In [23]:
gen_title_pop60.sort_values('Dom. Profit (millions)', ascending = False).head(10)
#sorting top ten for domestic profit to visualize  

,genre_ids,movie,popularity,Dom. Profit (millions),WW Profit (millions),first_genre_id
114,"[28, 18, 878]",Logan,45.000,$99.277068MM,$488.461394MM,Action
51,"[878, 28]",Venom,44.007,$97.511408MM,$737.628605MM,Sci Fi
108,"[28, 12, 878, 35]",Ant-Man and the Wasp,44.729,$86.64874MM,$493.14466MM,Action
306,"[36, 18, 53, 10752]",The Imitation Game,33.078,$76.125143MM,$212.740463MM,History
192,"[28, 9648, 878, 53]",The Maze Runner,36.955,$68.427862MM,$314.319861MM,Action
99,"[28, 12, 14, 878]",Doctor Strange,33.035,$67.64192MM,$511.404566MM,Action
98,"[28, 12, 14, 878]",Doctor Strange,33.035,$67.64192MM,$511.404566MM,Action
208,"[18, 35]",Green Book,36.284,$62.080171MM,$299.034439MM,Drama
119,"[12, 10751, 16, 28, 35]",Big Hero 6,36.920,$57.527828MM,$487.127828MM,Adventure
118,"[12, 10751, 16, 28, 35]",Big Hero 6,36.920,$57.527828MM,$487.127828MM,Adventure
